# Segmenting synchrotron x-radiographs with scikit-image

<center><h2>Marianne Corvellec</h2></center>
<center>
    <img src="images/skimage_logo.png" width="300">
    <img src="images/igdore_logo.png" width="300">
</center>

#### ALS User Meeting — August 12, 2021 ― Machine Learning for Synchrotrons

# Parent project: Quantifying solidification of metallic alloys with scikit-image

<center><h2>with C. Gus Becker</h2></center>
<center>
    <img src="images/csm_logo.png" width="400">
    <img src="images/canfsa_logo.png" width="300">
</center>

#### See our talk at BIDS ImageXD 2021:
#### https://www.youtube.com/watch?v=cB1HTgmWTd8

# Tracking the solid-liquid (S-L) interface

... to estimate the solidification velocity in a nickel alloy sample.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import imageio as io

In [ ]:
import plotly.express as px

In [ ]:
from skimage import measure, segmentation

#### https://scikit-image.org/

In [ ]:
collection = io.volread('data/nickel_solidification.tif')

In [ ]:
y1 = 0
y2 = 180
x1 = 100
x2 = 330

In [ ]:
collection = collection[:, y1:y2, x1:x2]

In [ ]:
collection.shape

In [ ]:
fig = px.imshow(collection, animation_frame=0, binary_string=True)
fig.show()

Imaging technique: Synchrotron x-radiography at the Advanced Photon Source (APS) of Argonne National Laboratory (ANL)

# Using the flood-fill algorithm

... to identify the liquid area.

In [ ]:
hline = 59
seed = pd.DataFrame({'y': [75], 'x': [110]})

In [ ]:
liquid = segmentation.flood(
    collection[0],
    seed_point=(seed.loc[0, 'y'], seed.loc[0, 'x']),
    tolerance=5000
)

In [ ]:
regions = measure.regionprops_table(measure.label(liquid))
regions

In [ ]:
fig = px.imshow(collection[0], color_continuous_scale='gray')
fig.add_hline(y=hline, line_color='blue')
fig.add_scatter(x=seed['x'], y=seed['y'])

In [ ]:
fig = px.imshow(liquid, color_continuous_scale='gray')
fig.show()

# Providing the series of seed points

... to start the flood fill.

In [ ]:
N = collection.shape[0]

In [ ]:
seeds = pd.DataFrame({'y': [75, 75, 75, 73, 73, 72, 72, 70, 70, 70],
                      'x': [110] * (N - 1)})

In [ ]:
seeds = pd.concat([seed, seeds], ignore_index=True)

What about using machine learning (gradient descent) to find seed points?

In [ ]:
fig = px.scatter(y=collection[0, :, seed.loc[0, 'x']])
fig.show()

In [ ]:
bounds = pd.DataFrame(np.zeros((N, 2)), columns=['upper', 'lower'])

In [ ]:
for i in range(N):
    liquid = segmentation.flood(
        collection[i],
        seed_point=(seeds.loc[i, 'y'], seeds.loc[i, 'x']),
        tolerance=5000
    )
    regions = measure.regionprops_table(measure.label(liquid))
    bounds.loc[i, 'upper'] = regions['bbox-0']
    bounds.loc[i, 'lower'] = regions['bbox-2']

In [ ]:
bounds

Sanity check for the upper bound (liquid area)

In [ ]:
fig = px.scatter(bounds, y='upper')
fig.add_hline(y=hline, line_color='blue')
fig.show()

In [ ]:
bounds['lower'] = bounds['lower'] - hline
fig = px.scatter(bounds, y='lower')
fig.show()

<center>
    <img src="images/prior_SL_detection.png" width="600">
</center>